## Data Mining Project - Olist Sentiment Analysis

### Loading Libraries and Datasets

In [33]:
# %pip install tensorflow
# %pip install -U scikit-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import sklearn

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [34]:
data_cust = pd.read_csv("datasets/olist_customers_dataset.csv")
data_order_item = pd.read_csv("datasets/olist_order_items_dataset.csv")
data_order_paym = pd.read_csv("datasets/olist_order_payments_dataset.csv")
data_order_review = pd.read_csv("datasets/olist_order_reviews_dataset.csv")
data_order_main = pd.read_csv("datasets/olist_orders_dataset.csv")
data_product = pd.read_csv("datasets/olist_products_dataset.csv")
data_sellers = pd.read_csv("datasets/olist_sellers_dataset.csv")
data_product_cate_name = pd.read_csv("datasets/product_category_name_translation.csv")

File Merger

In [52]:
compiled_df = data_cust.merge(data_order_main, how='inner', on='customer_id')
compiled_df = compiled_df.merge(data_order_review, how='inner', on='order_id')
compiled_df = compiled_df.merge(data_order_paym, how='inner', on='order_id')
compiled_df = compiled_df.merge(data_order_item, how='inner', on='order_id')
compiled_df = compiled_df.merge(data_product, how='inner', on='product_id')
compiled_df = compiled_df.merge(data_sellers, how='inner', on='seller_id')
compiled_df = compiled_df.merge(data_product_cate_name, how='inner', on='product_category_name')

In [58]:
compiled_df = compiled_df.dropna(subset=['order_delivered_carrier_date', 'order_delivered_customer_date',\
     'product_weight_g', 'product_length_cm', 'product_width_cm', 'product_height_cm']).reset_index()

Feature Selection

In [54]:
cols_to_drop = ['customer_id', 'customer_unique_id', 'customer_city', 'customer_state', 'order_id', 'review_id', 'review_score', 'review_comment_title', 'review_comment_message',\
     'review_creation_date', 'review_answer_timestamp', 'payment_sequential', 'payment_type', 'payment_installments', 'order_item_id', 'product_id',\
        'seller_id', 'seller_state', 'seller_city', 'product_category_name']
compiled_df.drop(cols_to_drop, axis=1, inplace=True)

In [56]:
compiled_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 115609 entries, 0 to 115608
Data columns (total 20 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   customer_zip_code_prefix       115609 non-null  int64  
 1   order_status                   115609 non-null  object 
 2   order_purchase_timestamp       115609 non-null  object 
 3   order_approved_at              115595 non-null  object 
 4   order_delivered_carrier_date   114414 non-null  object 
 5   order_delivered_customer_date  113209 non-null  object 
 6   order_estimated_delivery_date  115609 non-null  object 
 7   payment_value                  115609 non-null  float64
 8   shipping_limit_date            115609 non-null  object 
 9   price                          115609 non-null  float64
 10  freight_value                  115609 non-null  float64
 11  product_name_lenght            115609 non-null  float64
 12  product_description_lenght    